In [2]:
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from IPython.core.display import display, HTML

/Users/liuhaitian/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#Length of longest sentence
max_caption_len = 3
#Size of vocabulary 
vocab_size = 3

In [4]:
# Load one screenshot for each word, and turn them into digits 
images = []
for i in range(2):
    images.append(img_to_array(load_img('screenshot.jpg', target_size=(224, 224))))
images = np.array(images, dtype=float)
# Preprocess input for the VGG16 model
images = preprocess_input(images)

In [7]:
#Turn start tokens into one-hot encoding
html_input = np.array(
            [[[0., 0., 0.], #start
             [0., 0., 0.],
             [1., 0., 0.]],
             [[0., 0., 0.], #start <HTML>Hello World!</HTML>
             [1., 0., 0.],
             [0., 1., 0.]]])
#Turn next word into one-hot encoding
next_words = np.array(
            [[0., 1., 0.], # <HTML>Hello World!</HTML>
             [0., 0., 1.]]) # end

[[[0. 0. 0.]
  [0. 0. 0.]
  [1. 0. 0.]]

 [[0. 0. 0.]
  [1. 0. 0.]
  [0. 1. 0.]]]


In [5]:
# Load the VGG16 model trained on imagenet and output the classification feature
VGG = VGG16(weights='imagenet', include_top=True)
# Extract the features from the image
features = VGG.predict(images)

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 183s 0us/step


In [6]:
#Load the feature to the network, apply a dense layer, and repeat the vector
vgg_feature = Input(shape=(1000,))
vgg_feature_dense = Dense(5)(vgg_feature)
vgg_feature_repeat = RepeatVector(max_caption_len)(vgg_feature_dense)
# Extract information from the input seqence 
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(5, return_sequences=True)(language_input)

In [7]:
# Concatenate the information from the image and the input
decoder = concatenate([vgg_feature_repeat, language_model])
# Extract information from the concatenated output
decoder = LSTM(5, return_sequences=False)(decoder)
# Predict which word comes next
decoder_output = Dense(vocab_size, activation='softmax')(decoder)
# Compile and run the neural network
model = Model(inputs=[vgg_feature, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [8]:
# Train the neural network
model.fit([features, html_input], next_words, batch_size=2, shuffle=False, epochs=1000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
2/2 [==============================] - 1s 677ms/step - loss: 1.0975
Epoch 2/1000
2/2 [==============================] - 0s 4ms/step - loss: 1.0927
Epoch 3/1000
2/2 [==============================] - 0s 5ms/step - loss: 1.0892
Epoch 4/1000
2/2 [==============================] - 0s 6ms/step - loss: 1.0863
Epoch 5/1000
2/2 [==============================] - 0s 3ms/step - loss: 1.0836
Epoch 6/1000
2/2 [==============================] - 0s 5ms/step - loss: 1.0811
Epoch 7/1000
2/2 [==============================] - 0s 5ms/step - loss: 1.0788
Epoch 8/1000
2/2 [==============================] - 0s 3ms/step - loss: 1.0765
Epoch 9/1000
2/2 [==============================] - 0s 3ms/step - loss: 1.0742
Epoch 10/1000
2/2 [==============================] - 0s 3ms/step - loss: 1.0721
Epoch 11/1000
2/2 [==============================] - 0s 4ms/step - loss: 1.0699
Epoch 12/1000
2/2 [==============================] - 0s 5ms/step - loss: 1.0678

Epoch 100/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8669
Epoch 101/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8648
Epoch 102/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8626
Epoch 103/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.8604
Epoch 104/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8583
Epoch 105/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.8561
Epoch 106/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.8540
Epoch 107/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8518
Epoch 108/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8497
Epoch 109/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8475
Epoch 110/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8454
Epoch 111/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.8432
Epoch 112/1000
2/2 [========

Epoch 202/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6688
Epoch 203/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6669
Epoch 204/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6650
Epoch 205/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6631
Epoch 206/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6612
Epoch 207/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6593
Epoch 208/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.6574
Epoch 209/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6554
Epoch 210/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.6535
Epoch 211/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6515
Epoch 212/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6495
Epoch 213/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.6475
Epoch 214/1000
2/2 [========

Epoch 304/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.4014
Epoch 305/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3983
Epoch 306/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.3951
Epoch 307/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3923
Epoch 308/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3889
Epoch 309/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3859
Epoch 310/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.3828
Epoch 311/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3797
Epoch 312/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3766
Epoch 313/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.3734
Epoch 314/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3703
Epoch 315/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.3672
Epoch 316/1000
2/2 [========

2/2 [==============================] - 0s 4ms/step - loss: 0.1772
Epoch 406/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1757
Epoch 407/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1741
Epoch 408/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1727
Epoch 409/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1713
Epoch 410/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1698
Epoch 411/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1683
Epoch 412/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1669
Epoch 413/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1656
Epoch 414/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1641
Epoch 415/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1627
Epoch 416/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1613
Epoch 417/1000
2/2 [=======================

Epoch 507/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0733
Epoch 508/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0726
Epoch 509/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0720
Epoch 510/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0714
Epoch 511/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0708
Epoch 512/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0702
Epoch 513/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0695
Epoch 514/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0689
Epoch 515/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0683
Epoch 516/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0677
Epoch 517/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0671
Epoch 518/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0665
Epoch 519/1000
2/2 [========

2/2 [==============================] - 0s 3ms/step - loss: 0.0303
Epoch 609/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0300
Epoch 610/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0297
Epoch 611/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0295
Epoch 612/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0292
Epoch 613/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0290
Epoch 614/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0287
Epoch 615/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0285
Epoch 616/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0282
Epoch 617/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0280
Epoch 618/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0277
Epoch 619/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0275
Epoch 620/1000
2/2 [=======================

Epoch 710/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0128
Epoch 711/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0127
Epoch 712/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0126
Epoch 713/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0125
Epoch 714/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0124
Epoch 715/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0123
Epoch 716/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0122
Epoch 717/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0121
Epoch 718/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0120
Epoch 719/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0119
Epoch 720/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0118
Epoch 721/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0117
Epoch 722/1000
2/2 [========

2/2 [==============================] - 0s 4ms/step - loss: 0.0062
Epoch 812/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0061
Epoch 813/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0061
Epoch 814/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0060
Epoch 815/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0060
Epoch 816/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0060
Epoch 817/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0059
Epoch 818/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0059
Epoch 819/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0058
Epoch 820/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0058
Epoch 821/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0058
Epoch 822/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0057
Epoch 823/1000
2/2 [=======================

Epoch 913/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 914/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0031
Epoch 915/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0031
Epoch 916/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0031
Epoch 917/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 918/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0031
Epoch 919/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0030
Epoch 920/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0030
Epoch 921/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0030
Epoch 922/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.0030
Epoch 923/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0030
Epoch 924/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0030
Epoch 925/1000
2/2 [========

In [9]:
start_token = [1., 0., 0.] # start
sentence = np.zeros((1, 3, 3)) # [[0,0,0], [0,0,0], [0,0,0]]
sentence[0][2] = start_token # place start in empty sentence

In [10]:
# Making the first prediction with the start token
second_word = model.predict([np.array([features[1]]), sentence])

In [11]:
# Put the second word in the sentence and make the final prediction
sentence[0][1] = start_token
sentence[0][2] = np.round(second_word)X
third_word = model.predict([np.array([features[1]]), sentence])

In [12]:
# Place the start token and our two predictions in the sentence 

In [13]:
sentence[0][0] = start_token
sentence[0][1] = np.round(second_word)
sentence[0][2] = np.round(third_word)

In [14]:
# Transform our one-hot predictions into the final tokens
vocabulary = ["start", "<HTML><center><H1>Hello World!</H1><center></HTML>", "end"]
html = ""
for i in sentence[0]:
    html += vocabulary[np.argmax(i)] + ' '

In [15]:
from IPython.core.display import display, HTML
display(HTML(html[6:49]))

In [16]:
html

'start <HTML><center><H1>Hello World!</H1><center></HTML> end '